# What is Filtering in DRF?

Filtering allows clients to retrieve a subset of data by passing parameters in the query string — e.g., `/api/products?category=books`.

**Filtering enhances API flexibility and usability by letting users query data more precisely.**

# 1. Generic Filtering (Basic)

**Built-in DRF filtering using `filter_backends` + `filterset_fields`**

- DRF supports basic filtering using query parameters
- No extra package is needed

# 2. Django Filter Backend (Advanced Filtering)

**What is django-filter?**

`django-filter` is a third-party Django package that:
- Automates filtering interfaces
- Supports complex lookups (e.g., greater than, contains)
- Integrates with DRF's ViewSet

**Why use `django-filter` with DRF?**

- Enables filtering by price ranges, substrings, dates, foreign keys
- Integrates with `ModelViewSet`
- Example: `/api/products/?min_price=100&max_price=500`

## Step 1: Install django-filter

In [ ]:
!pip install django-filter

## Step 2: Update settings.py

In [ ]:
INSTALLED_APPS = [
    ...,
    'django_filters',
]

REST_FRAMEWORK = {
    'DEFAULT_FILTER_BACKENDS': [
        'django_filters.rest_framework.DjangoFilterBackend'
    ]
}

## Step 3: Use in a ViewSet (Simple Filtering)

In [ ]:
from rest_framework import viewsets
from django_filters.rest_framework import DjangoFilterBackend
from .models import Product
from .serializers import ProductSerializer

class ProductViewSet(viewsets.ModelViewSet):
    queryset = Product.objects.all()
    serializer_class = ProductSerializer
    filter_backends = [DjangoFilterBackend]
    filterset_fields = ['category', 'brand']

**Example API calls:**
- `/api/products/?category=books`
- `/api/products/?brand=sony`

# 4. Custom Filtering with FilterSet Class (Advanced)

In [ ]:
# filters.py
import django_filters
from .models import Product

class ProductFilter(django_filters.FilterSet):
    min_price = django_filters.NumberFilter(field_name='price', lookup_expr='gte')
    max_price = django_filters.NumberFilter(field_name='price', lookup_expr='lte')
    brand = django_filters.CharFilter(lookup_expr='icontains')

    class Meta:
        model = Product
        fields = ['category', 'brand']

In [ ]:
# views.py
from .filters import ProductFilter
from django_filters.rest_framework import DjangoFilterBackend

class ProductViewSet(viewsets.ModelViewSet):
    queryset = Product.objects.all()
    serializer_class = ProductSerializer
    filter_backends = [DjangoFilterBackend]
    filterset_class = ProductFilter

**Example Queries:**
- `/api/products/?min_price=500&max_price=1500`
- `/api/products/?brand=sony`

# 5. Django ORM `.filter()` Examples

In [ ]:
from django.db import models

class Student(models.Model):
    name = models.CharField(max_length=100)
    age = models.IntegerField()
    grade = models.CharField(max_length=5)

In [ ]:
# Example Queries
Student.objects.filter(name='John')                        # Exact match
Student.objects.filter(age__gt=18)                         # Greater than
Student.objects.filter(grade__in=['A', 'A+'])              # In list
Student.objects.filter(name__icontains='an')              # Case-insensitive substring
Student.objects.filter(age__gte=18, grade='A')            # Multiple conditions

# 6. filter() vs get()

| Method | Returns              | Use Case               |
|--------|----------------------|-------------------------|
| filter() | QuerySet (list)     | Multiple or no results |
| get()    | Single object/error | Exactly one result     |

# ✅ Summary Table

| Feature                | Generic Filtering | django-filter        |
|------------------------|-------------------|----------------------|
| Installation Needed    | ❌ No              | ✅ Yes               |
| Simple field filtering | ✅ Yes             | ✅ Yes               |
| Complex/custom filters | ❌ No              | ✅ Yes               |
| Recommended for prod   | ❌ No              | ✅ Yes               |